### Let's implement and test the simplest flow
1. User asks a question
2. Agent decides which tool to use
3. Agent uses the tool
4. Agent returns the answer to the user

### Minimal imports

In [ ]:
import os
import requests
import json
from dotenv import load_dotenv
load_dotenv()

### User's message

In [ ]:
user_message = "What's the weather in Warsaw? What can I do there with that weather?"

### Tools Definition

In [ ]:
check_weather_name = "get_weather"
check_weather_description = "Get the current weather in a given location"

In [ ]:
wikipedia_facts_name = "facts_from_wikipedia"
wikipedia_facts_description = "Get the wikipedia facts about a given topic, location, person etc."

#### (A) Anthropic Tools Definition

In [ ]:
check_weather_tool_anthropic = {
    "name": check_weather_name,
    "description": check_weather_description,
    "input_schema": {
        "type": "object",
        "properties": {
            "latitude": {
                "type": "string",
                "description": "latitude of the location for which to get the weather",
            },
            "longitude": {
                "type": "string",
                "description": "longitude of the location for which to get the weather",
            },
        },
        "required": ["latitude", "longitude"],
    },
}

In [ ]:
wikipedia_facts_tool_anthropic = {
    "name": wikipedia_facts_name,
    "description": wikipedia_facts_description,
    "input_schema": {
        "type": "object",
        "properties": {
            "topic": {
                "type": "string",
                "description": "A topic, location, person etc. to search for on Wikipedia",
            },
        },
        "required": ["topic"],
    },
}

In [ ]:
tools_antrhopic = [check_weather_tool_anthropic]
tools_antrhopic.append(wikipedia_facts_tool_anthropic)

#### (B) OpenAI Tools Definition

In [ ]:
check_weather_tool_openai = {
    "type": "function",
    "name": check_weather_name,
    "description": check_weather_description,
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {
                "type": "string",
                "description": "latitude of the location for which to get the weather",
            },
            "longitude": {
                "type": "string",
                "description": "longitude of the location for which to get the weather",
            },
        },
        "required": ["latitude", "longitude"],
        "additionalProperties": False,  # Additional keys are not allowed

    },
    "strict": True,  # Enforce strict adherence to the schema
}

In [ ]:
wikipedia_facts_tool_openai = {
    "type": "function",
    "name": wikipedia_facts_name,
    "description": wikipedia_facts_description,
    "parameters": {
        "type": "object",
        "properties": {
            "topic": {
                "type": "string",
                "description": "A topic, location, person etc. to search for on Wikipedia",
            },
        },
        "required": ["topic"],
        "additionalProperties": False,
    },
    "strict": True,
}

In [ ]:
tools_openai = [check_weather_tool_openai]
tools_openai.append(wikipedia_facts_tool_openai)

### Let's make the first LLM call to decide which tool to use and with what parameters

#### (A) Anthropic LLM Call

In [ ]:
url_anthropic = "https://api.anthropic.com/v1/messages"

headers_anthropic = {
    "Content-Type": "application/json",
    "x-api-key": os.getenv("ANTHROPIC_API_KEY"),
    "anthropic-version": "2023-06-01",
}

In [ ]:
payload_anthropic = {
    "model": "claude-sonnet-4-5",
    "max_tokens": 300,
    "tools": tools_antrhopic,
    "tool_choice": {"type": "auto"},
    "messages": [
        {
            "role": "user",
            "content": user_message,
        }
    ]
}
resp_anthropic = requests.post(url_anthropic, headers=headers_anthropic, json=payload_anthropic)
print(resp_anthropic.status_code)

In [ ]:
response_content_anthropic = resp_anthropic.json().get("content", [])

#### (B) OpenAI LLM Call

In [ ]:
openai_direct_config = {
    "url": "https://api.openai.com/v1/responses",
    "api_key": os.getenv("OPENAI_API_KEY"),
    "model": "gpt-5",
    }

openai_azure_config = {
    "url": f"https://{os.getenv("DEV_AZURE_RESOURCE")}.openai.azure.com/openai/v1/responses",
    "api_key": os.getenv("DEV_AZURE_API_KEY"),
    "model": os.getenv("DEV_AZURE_OPENAI_MODEL"),
}

openai_config = openai_azure_config

headers_openai = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {openai_config['api_key']}",
}

In [ ]:
payload_openai = {
    "model": openai_config["model"],
    "tools": tools_openai,
    "input": user_message,
    # "reasoning": {
    #     "effort": "medium",
    #     "summary": "auto"
    # },
}

resp_openai = requests.post(openai_config["url"], headers=headers_openai, json=payload_openai)
print(resp_openai.status_code)

In [ ]:
response_content_openai = resp_openai.json().get("output", [])

### Now let's process the output from the tool use step

#### (A) Extracting tools from Anthropic response

In [ ]:
tools = [block for block in response_content_anthropic if block.get("type") == "tool_use"]

In [ ]:
for tool in tools:
    if tool["name"] == "get_weather":
        latitude = tool["input"]["latitude"]
        longitude = tool["input"]["longitude"]
        weather_tool_response = requests.get(
            url="https://api.open-meteo.com/v1/forecast",
            params={"latitude": latitude, "longitude": longitude, "current_weather": True},
            # verify=False
        )
        weather_info = weather_tool_response.json()["current_weather"]
        tool["output"] = weather_info
    if tool["name"] == "facts_from_wikipedia":
        topic = tool["input"]["topic"]
        headers = {
            'User-Agent': 'Agent101Bot/1.0 (Educational purpose only)',
            'Accept': 'application/json'
        }
        wikipedia_response = requests.get(
            url=f"https://en.wikipedia.org/w/api.php?action=query&prop=revisions&titles={topic}&rvprop=content&format=json",
            headers=headers,
            # verify=False
        )
        wiki_info = wikipedia_response.json()
        tool["output"] = wiki_info

#### (B) Extracting tools from OpenAI response

In [ ]:
tools = [block for block in response_content_openai if block.get("type") == "function_call"]

In [ ]:
for tool in tools:
    if tool["name"] == "get_weather":
        latitude = json.loads(tool["arguments"])["latitude"]
        longitude = json.loads(tool["arguments"])["longitude"]
        weather_tool_response = requests.get(
            url="https://api.open-meteo.com/v1/forecast",
            params={"latitude": latitude, "longitude": longitude, "current_weather": True},
            verify=False # simpler than adding TMPL certs in this demo
        )
        weather_info = weather_tool_response.json()["current_weather"]
        tool["output"] = weather_info
    if tool["name"] == "facts_from_wikipedia":
        topic = latitude = json.loads(tool["arguments"])["topic"]
        headers = {
            'User-Agent': 'Agent101Bot/1.0 (Educational purpose only)',
            'Accept': 'application/json'
        }
        wikipedia_response = requests.get(
            url=f"https://en.wikipedia.org/w/api.php?action=query&prop=revisions&titles={topic}&rvprop=content&format=json",
            headers=headers,
            verify=False
        )
        wiki_info = wikipedia_response.json()
        tool["output"] = wiki_info

#### Collecting outputs from all tools into a single string

In [ ]:
tool_outputs = "\n".join([f"Tool: {tool['name']}, Output: {tool.get('output', 'No output')}" for tool in tools])

### Finally, let's make the last LLM call to get the final answer

In [ ]:
final_prompt = f"""You are an assistant that responds to user query based on the tools used. Respond politely to user's query using only the output from the tools. If the tool_output is empty, then always respond with "I'm not able to answer". Use only information from the tool_outputs to answer the question. Do not mention anything about the the tools. If information necessary to answer some of the questions or part of a question is missing from the tool_outputs, say you're not able to answer to that particular part.

<user_query>
{user_message}
</user_query>

<tool_outputs>
{tool_outputs}
</tool_outputs>

Remember, to answer the questions in <user_query>, use the information from the <tool_outputs> section only. If some information is missing from the <tool_outputs>, do not reply to that specific part of the query.
"""

#### (A) Anthropic final LLM call

In [ ]:
payload_final_anthropic = {
    "model": "claude-sonnet-4-5",
    "max_tokens": 300,
    "messages": [
        {
            "role": "user",
            "content": final_prompt,
        }
    ]
}
response_final_anthropic = requests.post(url_anthropic, headers=headers_anthropic, json=payload_final_anthropic)
print(response_final_anthropic.status_code)

In [ ]:
final_response = response_final_anthropic.json().get("content", [])[0].get("text", "No response")

#### (B) OpenAI final LLM call

In [ ]:
payload_final_openai = {
    "model": openai_config["model"],
    "input": final_prompt,
}
response_final_openai = requests.post(openai_config["url"], headers=headers_openai, json=payload_final_openai)
print(response_final_openai.status_code)

In [ ]:
final_response = response_final_openai.json().get("output", "No response")[1]["content"][0]["text"]

### Reading the final response

In [ ]:
print(final_response)